In [ ]:
# %% @title Environment Setup - MNIST
!pip install -q wandb tensorflow==2.15.0
import tensorflow as tf
from tensorflow.keras import layers, models
import wandb
from wandb.integration.keras import WandbCallback
import numpy as np
import matplotlib.pyplot as plt

# %%
# Load and preprocess MNIST data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0
test_images  = test_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0



# %%
def create_mnist_model():
    model = models.Sequential(name="sequential_1")
    model.add(layers.Flatten(input_shape=(28, 28), name="flatten_1"))
    model.add(layers.Dense(128, activation="relu", name="dense_2"))
    model.add(layers.Dropout(0.2, name="dropout_1"))
    model.add(layers.Dense(10, activation="softmax", name="dense_3"))
    return model

mnist_model = create_mnist_model()
mnist_model.summary()  # This prints the model summary with box‐drawing characters and ANSI colors




# %%
detailed_summary = tf.keras.utils.model_to_dot(mnist_model, show_shapes=True).create(prog='dot', format='plain')
print("Detailed Model Summary:")
print(detailed_summary)




# %%
wandb.init(project="mnist-classification", reinit=True)
wandb.config.update({"learning_rate": 0.001, "epochs": 15, "batch_size": 128})
mnist_model.compile(optimizer="adam",
                     loss="sparse_categorical_crossentropy",
                     metrics=["accuracy"])
history_mnist = mnist_model.fit(train_images, train_labels,
                                validation_split=0.2,
                                epochs=15,
                                batch_size=128,
                                callbacks=[WandbCallback()])



# %%
test_loss, test_acc = mnist_model.evaluate(test_images, test_labels)
y_pred_mnist = mnist_model.predict(test_images).argmax(axis=1)

# Log confusion matrix using W&B
wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(
    y_true=test_labels,
    preds=y_pred_mnist,
    class_names=[str(i) for i in range(10)]
)})

# Log a few error examples (misclassified samples)
misclassified = (y_pred_mnist != test_labels.flatten())
error_samples = list(zip(test_images[misclassified], y_pred_mnist[misclassified], test_labels[misclassified]))[:5]
wandb.log({"error_examples": [
    wandb.Image(img, caption=f"Pred: {p}, True: {t}")
    for img, p, t in error_samples
]})

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.71 requires ml_dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.15.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.15.0 which is incompatible.
11490434/11490434 [=======================

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jayanth-kalyanam (jayanth-kalyanam-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/15
365/375 [============================>.] - ETA: 0s - loss: 0.4503 - accuracy: 0.8713

...

375/375 [==============================] - 2s 6ms/step - loss: 0.1111 - accuracy: 0.9671 - val_loss: 0.0999 - val_accuracy: 0.9700
Epoch 6/15
362/375 [===========================>..] - ETA: 0s - loss: 0.0981 - accuracy: 0.9707

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20250220_072233-6gyynygc/files/model-best)... Done. 0.0s


375/375 [==============================] - 3s 7ms/step - loss: 0.0980 - accuracy: 0.9708 - val_loss: 0.0925 - val_accuracy: 0.9725
Epoch 7/15
371/375 [============================>.] - ETA: 0s - loss: 0.0859 - accuracy: 0.9743

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20250220_072233-6gyynygc/files/model-best)... Done. 0.0s


375/375 [==============================] - 3s 9ms/step - loss: 0.0857 - accuracy: 0.9743 - val_loss: 0.0878 - val_accuracy: 0.9752
Epoch 8/15
270/375 [====================>.........] - ETA: 0s - loss: 0.0756 - accuracy: 0.9768